<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# train.py (minimal pattern)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self, in_dim=32, hidden=64, out_dim=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden), nn.ReLU(),
            nn.Linear(hidden, hidden), nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x): return self.net(x)

def main(args=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Toy data (replace with your dataset)
    X = torch.randn(4096, 32)
    y = torch.randint(0, 10, (4096,))
    ds = TensorDataset(X, y)
    dl = DataLoader(ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)

    model = Net().to(device)
    try:
        model = torch.compile(model, fullgraph=True)  # PyTorch 2.x optional speed-up
    except Exception:
        pass

    opt = optim.AdamW(model.parameters(), lr=5e-4)
    loss_fn = nn.CrossEntropyLoss()

    epochs = 5
    model.train()
    for epoch in range(epochs):
        for xb, yb in dl:
            xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
            logits = model(xb)
            loss = loss_fn(logits, yb)
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()
        print(f"epoch {epoch+1}: loss={loss.item():.4f}")

if __name__ == "__main__":
    main()